In [ ]:
#
# Copyright (C) 2018 Franz Elsner <felsner[AT]mpa-garching.mpg.de> 
#
# Provide basic CMB analysis functions
#
# Press Shift+Enter to execute cells
#

from numpy import *
import healpy as hp
import camb4py
import matplotlib.pyplot as plt
from pylab import cm as colormap
from os import getenv
from os.path import join

%matplotlib qt
file_cmb_map   = 'data/storage/files/planck_2015/smica_lowres_nobeam.fits'
file_camb_exec = 'applications/camb/camb'
file_camb_ini  = 'applications/camb/planck15_ttteee_lowteb_lensing.cfg'

def load_cmb_data():
    file_name = join(getenv('HOME'), file_cmb_map)
    skymap    = hp.read_map(file_name)
    return skymap

def plot_cmb_map(skymap, figure=1, title=''):
    fig = plt.figure(figure)
    fig.clf()
    cmap = colormap.jet
    cmap.set_under('w')
    hp.mollzoom(skymap, min=-0.0005, max=0.0005, fig=figure, title=title, unit='Kelvin', cmap=cmap)
    return

def multipole2angle_ticks(l):
    theta = 180.0 / l
    return ["{0:4.1f}".format(t) for t in theta]

def plot_power_spectrum(skymap, figure=1, title='', lmax=1000):
    l   = arange(lmax+1)
    cls = hp.anafast(skymap, lmax=lmax, iter=1)
    fig = plt.figure(figure)
    fig.suptitle(title)
    ax1 = fig.add_subplot(1, 1, 1)
    ax1.plot(l, l*(l+1)/(2.0*pi) * 1.0E12 * cls)
    ax1.yaxis.set_major_locator(plt.MaxNLocator(nbins=4))
    ax1.tick_params(axis='both', which='major', labelsize=16)
    ax1.set_xlim([0, lmax])
    ax1.set_ylim([0, ax1.get_ylim()[1]])
    ax1.set_xlabel(r'$\ell$', fontsize=20)
    ax1.set_ylabel(r'$\ell(\ell+1)/2\pi \cdot C_{\ell} \ [\mu K^2]$', fontsize=20)
    l_at_theta = 180.0/array([90.0, 1.0, 0.5, 0.2])
    ax2 = ax1.twiny()
    ax2.set_xlim(ax1.get_xlim())
    ax2.set_xticks(l_at_theta)
    ax2.tick_params(axis='both', which='major', labelsize=16)
    ax2.set_xticklabels(multipole2angle_ticks(l_at_theta))
    ax2.set_xlabel(r"$\mathrm{Angle} \ [^{\circ}]$", fontsize=20)
    fig.tight_layout()
    return

class boltzmann_solver(object):
    def __init__(self, camb_executable, camb_parameters, lmax=1000, nside=512):
        file_exec   = join(getenv('HOME'), camb_executable)
        file_par    = join(getenv('HOME'), camb_parameters)
        self.lmax   = lmax
        self.nside  = nside
        self.camb   = camb4py.load(file_exec, defaults=file_par)
        self.l      = arange(2, self.lmax + 1)
        self.norm   = 2.0*pi/(self.l*(self.l+1.0)) * 2.7255**2
        self.cl_ref = self.camb()['scalar'][:self.lmax-1,1]
        return
    
    def rescale_alms(self, cosmology):
        cl_new = self.camb(**cosmology)['scalar'][:self.lmax-1,1]
        factor = sqrt(hstack([ones(2), cl_new / self.cl_ref]))
        return factor

    def rescale_cmb_map(self, cmb_map, cosmology):
        valid_keys = ["scalar_spectral_index(1)", "hubble", "ombh2", "omch2"]
        for key in cosmology.keys():
            if not key in valid_keys:
                print 'Invalid entry "{0:}"'.format(key)
                print 'Did you mean any of {0:}?'.format(valid_keys)
                return None
        alms   = hp.map2alm(cmb_map, lmax=self.lmax)
        factor = self.rescale_alms(cosmology)
        hp.almxfl(alms, factor, inplace=True)
        rescaled_map = hp.alm2map(alms, self.nside);
        return rescaled_map
    
    def simulate_cmb_map(self):
        cl_ref = hstack([zeros(2), self.norm * self.cl_ref])
        simulated_map = hp.synfast(cl_ref, self.nside, new=True)
        return simulated_map

camb = boltzmann_solver(file_camb_exec, file_camb_ini)